In [7]:
import numpy as np


In [8]:
with open("sample.dat", "r") as f:
    lines = f.readlines()


for l in lines[1:8]:
    print(l[:-1].split(":"))


for l in lines[9:20]:
    print(l.split())


['VERTICES ', ' 8']
['DEPOT ', ' 1']
['REQUIRED EDGES ', ' 5']
['NON-REQUIRED EDGES ', ' 5']
['VEHICLES ', ' 2']
['CAPACITY ', ' 6']
['TOTAL COST OF REQUIRED EDGES ', ' 13']
['1', '2', '4', '0']
['2', '3', '2', '3']
['3', '4', '3', '3']
['1', '4', '4', '0']
['4', '5', '7', '0']
['5', '6', '2', '2']
['6', '7', '3', '2']
['7', '8', '3', '2']
['1', '8', '1', '0']
['1', '5', '3', '0']
['END']


In [9]:
MAX_COST = int(1e9)

def read_file(path):
    with open(path, "r") as f:
        contents = f.readlines()
    num_vertices = int(contents[1].split(":")[1])
    depot = int(contents[2].split(":")[1])
    num_required_edges = int(contents[3].split(":")[1])
    num_non_required_edges = int(contents[4].split(":")[1])
    num_vehicles = int(contents[5].split(":")[1])
    capacity = int(contents[6].split(":")[1])
    total_cost = int(contents[7].split(":")[1])

    graph = np.zeros((num_vertices, num_vertices), dtype=int)
    graph.fill(MAX_COST)
    tasks = np.zeros((num_vertices, num_vertices), dtype=int)

    for c in contents[9: 9 + num_required_edges + num_non_required_edges]:
        c = [int(i) for i in c.split()]
        n1, n2 = c[0] - 1, c[1] - 1
        graph[n1, n2] = c[2]
        graph[n2, n1] = c[2]
        if c[3] != 0:
            tasks[n1, n2] = c[3]
            tasks[n2, n1] = c[3]


    info = {
        "num_vertices": num_vertices,
        "depot": depot,
        "num_required_edges": num_required_edges,
        "num_non_required_edges": num_non_required_edges,
        "num_vehicles": num_vehicles,
        "capacity": capacity,
        "total_cost": total_cost,
        "graph": graph,
        "tasks": tasks
    }
    return info


In [10]:
read_file("sample.dat")

{'num_vertices': 8,
 'depot': 1,
 'num_required_edges': 5,
 'num_non_required_edges': 5,
 'num_vehicles': 2,
 'capacity': 6,
 'total_cost': 13,
 'graph': array([[1000000000,          4, 1000000000,          4,          3,
         1000000000, 1000000000,          1],
        [         4, 1000000000,          2, 1000000000, 1000000000,
         1000000000, 1000000000, 1000000000],
        [1000000000,          2, 1000000000,          3, 1000000000,
         1000000000, 1000000000, 1000000000],
        [         4, 1000000000,          3, 1000000000,          7,
         1000000000, 1000000000, 1000000000],
        [         3, 1000000000, 1000000000,          7, 1000000000,
                  2, 1000000000, 1000000000],
        [1000000000, 1000000000, 1000000000, 1000000000,          2,
         1000000000,          3, 1000000000],
        [1000000000, 1000000000, 1000000000, 1000000000, 1000000000,
                  3, 1000000000,          3],
        [         1, 1000000000, 100000000

In [24]:
from heapq import heapify, heappop, heappush
def shortest_path(graph):
    n = graph.shape[0]
    sp = np.zeros_like(graph, dtype=int)
    sp.fill(1e9)
    is_visited = np.zeros((n, ), dtype=bool)

    for src in range(n):
        is_visited.fill(False)
        heap = [(0, src)]  # (distance, point)
        sp[src, src] = 0  # 

        while len(heap) > 0:
            cost, s = heappop(heap)
            if is_visited[s]: continue
            is_visited[s] = True


            for d in range(n):
                # 没有被访问过
                new_cost = cost + graph[s, d]
                if not is_visited[d] and new_cost < sp[src, d]:
                    sp[src, d] = new_cost
                    
                    # 将d插入堆中
                    heappush(heap, (new_cost, d))

    return sp


In [25]:
graph = read_file("sample.dat")["graph"]
shortest_path(graph)

array([[ 0,  4,  6,  4,  3,  5,  4,  1],
       [ 4,  0,  2,  5,  7,  9,  8,  5],
       [ 6,  2,  0,  3,  9, 11, 10,  7],
       [ 4,  5,  3,  0,  7,  9,  8,  5],
       [ 3,  7,  9,  7,  0,  2,  5,  4],
       [ 5,  9, 11,  9,  2,  0,  3,  6],
       [ 4,  8, 10,  8,  5,  3,  0,  3],
       [ 1,  5,  7,  5,  4,  6,  3,  0]])